In [3]:
from joblib import load
from os import path

In [8]:
dummy = True

In [9]:
task_name = "crabnet_hyperparameter"
model_dir = path.join("..", "..", "models", task_name)

if dummy:
    model_dir = path.join(model_dir, "dummy")
   

In [16]:
class CrabNetSurrogateModel(object):
    def __init__(self):
        self.models = load(path.join(model_dir, "surrogate_models.pkl"))

    def prepare_params_for_eval():


    
    def surrogate_evaluate(self, parameters):
        
        mae = self.models["mae"].predict(parameters)
        rmse = self.models["rmse"].predict(parameters)
        runtime = self.models["rmse"].predict(parameters)
        model_size = self.models["model_size"].predict(parameters)

        #somehow incoroprate the rank/percentile
        
        return mae, rmse, runtime, model_size



In [12]:
c = CrabNetSurrogateModel()

In [17]:
def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df
